In [1]:
import requests
import json
import torch
import os
from tqdm import tqdm

In [2]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.7 MB/s eta 0:00:00


In [4]:
with open('spoken_train-v1.1.json', 'rb') as f:
  squad = json.load(f)

In [5]:
squad['data'][0].keys()


dict_keys(['title', 'paragraphs'])

In [6]:
gr = -1
for idx, group in enumerate(squad['data']):
  print(group['title'])
  if group['title'] == 'Greece':
    gr = idx
    print(gr)
    break

University_of_Notre_Dame
Beyoncé
Montana
Genocide
Antibiotics
Frédéric_Chopin
Sino-Tibetan_relations_during_the_Ming_dynasty
IPod
The_Legend_of_Zelda:_Twilight_Princess
Spectre_(2015_film)
2008_Sichuan_earthquake
New_York_City
To_Kill_a_Mockingbird
Solar_energy
Tajikistan
Anthropology
Portugal
Kanye_West
Buddhism
American_Idol
Dog
2008_Summer_Olympics_torch_relay
Alfred_North_Whitehead
Financial_crisis_of_2007%E2%80%9308
Saint_Barth%C3%A9lemy
Genome
Comprehensive_school
Republic_of_the_Congo
Prime_minister
Institute_of_technology
Wayback_Machine
Dutch_Republic
Symbiosis
Canadian_Armed_Forces
Cardinal_(Catholicism)
Iranian_languages
Lighting
Separation_of_powers_under_the_United_States_Constitution
Architecture
Human_Development_Index
Southern_Europe
BBC_Television
Arnold_Schwarzenegger
Plymouth
Heresy
Warsaw_Pact
Materialism
Space_Race
Pub
Christian
Sony_Music_Entertainment
Oklahoma_City
Hunter-gatherer
United_Nations_Population_Fund
Russian_Soviet_Federative_Socialist_Republic
Univers

In [7]:
squad['data'][186]['paragraphs'][0]['context']


'napoleon bonaparte nato mean l l j e n french nap led the napa tea or nepali on deep wanna parte the fifteenth of august seventeen sixty nine to the fifth of may eighteen twenty one was a french military and political leader who rose to prominence during the french revolution and led several successful campaigns during the revolutionary war sir. as napoleon i he was emperor of the french from eighty know for intel eighteen fourteen and again in eighteen fifteen. napoleon dominated european in global affairs for more than a decade while leaving france against a series of coalitions in the napoleonic wars. he won most of these wars and the vast majority of his battles building a large empire that ruled over continental europe before its final collapse in eighteen fifteen. often considered one of the greatest commanders in history his wars in campaigns are studied at military schools worldwide. he also remains one of the most celebrated and controversial political figures in western hist

In [8]:
def read_data(path):  
  # load the json file
  with open(path, 'rb') as f:
    squad = json.load(f)

  contexts = []
  questions = []
  answers = []

  for group in squad['data']:
    for passage in group['paragraphs']:
      context = passage['context']
      for qa in passage['qas']:
        question = qa['question']
        for answer in qa['answers']:
          contexts.append(context)
          questions.append(question)
          answers.append(answer)

  return contexts, questions, answers

In [ ]:
# train_contexts, train_questions, train_answers = read_data('train_data.json')
# valid_contexts, valid_questions, valid_answers = read_data('val_data.json')

In [9]:
train_contexts, train_questions, train_answers = read_data('spoken_train-v1.1.json')
valid_contexts, valid_questions, valid_answers = read_data('spoken_test-v1.1.json')
     


In [10]:
print(f'There are {len(train_questions)} questions')
print(train_questions[-10000])
print(train_answers[-10000])
     

There are 37111 questions
What country borders south Estonia?
{'answer_start': 262, 'text': 'latvia'}


In [11]:
def add_end_idx(answers, contexts):
  for answer, context in zip(answers, contexts):
    gold_text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(gold_text)

    if context[start_idx:end_idx] == gold_text:
      answer['answer_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
      answer['answer_start'] = start_idx - 1
      answer['answer_end'] = end_idx - 1     
    elif context[start_idx-2:end_idx-2] == gold_text:
      answer['answer_start'] = start_idx - 2
      answer['answer_end'] = end_idx - 2    

add_end_idx(train_answers, train_contexts)
add_end_idx(valid_answers, valid_contexts)

In [12]:
print(train_questions[-10000])
print(train_answers[-10000])

What country borders south Estonia?
{'answer_start': 262, 'text': 'latvia', 'answer_end': 268}


In [13]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

doc_stride = 128  

train_encodings = tokenizer(
    train_contexts,
    train_questions,
    truncation=True,
    padding=True,
    max_length=512,
    stride=doc_stride
)

valid_encodings = tokenizer(
    valid_contexts,
    valid_questions,
    truncation=True,
    padding=True,
    max_length=512,
    stride=doc_stride
)

In [14]:
train_encodings.keys()


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [15]:
no_of_encodings = len(train_encodings['input_ids'])
print(f'We have {no_of_encodings} context-question pairs')
     

We have 37111 context-question pairs


In [16]:
tokenizer.decode(train_encodings['input_ids'][0])


'[CLS] architecturally the school has a catholic character. atop the main building school dome is the golden statue of the virgin mary. immediately in front of the main building in facing it is a copper statue of christ with arms appraised with the legend and the bad meow names. next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto im mary in place of prayer and reflection. it is a replica of the grotto at lourdes france where the virgin mary reputedly appeared to st bernadette still burning eighteen fifty eight. at the end of the main drive and in a direct line that connects through three statues in the gold dome is as simple modern stone statue of mary. [SEP] what is in front of the notre dame main building? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [17]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i, answer in enumerate(answers):
        start_pos = max(0, answer['answer_start'])
        end_pos = max(0, answer['answer_end'] - 1)
        start_positions.append(encodings.char_to_token(i, start_pos))
        end_positions.append(encodings.char_to_token(i, end_pos))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(valid_encodings, valid_answers)


In [18]:
train_encodings['start_positions'][:10]


[36, 11, 55, 107, 25, 21, 43, 67, 28, 22]

In [19]:
class SQuAD_Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)
     

In [20]:

train_dataset = SQuAD_Dataset(train_encodings)
valid_dataset = SQuAD_Dataset(valid_encodings)
     

In [21]:
from torch.utils.data import DataLoader

# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16)

In [22]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [23]:

# Check on the available device - use GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')
     

Working on cuda


In [26]:
from transformers import AdamW, get_linear_schedule_with_warmup

N_EPOCHS = 1
MAX_LR = 5e-5
WARMUP_STEPS = 100
TOTAL_STEPS = len(train_loader) * N_EPOCHS
GRADIENT_ACCUMULATION_STEPS = 8

optim = AdamW(model.parameters(), lr=MAX_LR)

scheduler = get_linear_schedule_with_warmup(
    optim, num_warmup_steps=WARMUP_STEPS, num_training_steps=TOTAL_STEPS
)

model.to(device)
model.train()
i=0
for epoch in range(N_EPOCHS):
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]

        loss = loss / GRADIENT_ACCUMULATION_STEPS
        loss.backward()

        if (i + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optim.step()
            scheduler.step()
            optim.zero_grad()

        loop.set_description(f'Epoch {epoch+1}')
        loop.set_postfix(loss=loss.item())


Epoch 1: 100%|██████████| 2320/2320 [1:01:25<00:00,  1.59s/it, loss=0.795]


In [28]:
!pip install jiwer
     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.5 MB/s eta 0:00:00


In [30]:
def get_prediction(context, question):
  inputs = tokenizer.encode_plus(question, context, return_tensors='pt').to(device)
  outputs = model(**inputs)
  
  answer_start = torch.argmax(outputs[0])  
  answer_end = torch.argmax(outputs[1]) + 1 
  
  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
  
  return answer

def normalize_text(s):
  """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
  import string, re
  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)
  def white_space_fix(text):
    return " ".join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def exact_match(prediction, truth):
    return bool(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()
  
  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)
  
  common_tokens = set(pred_tokens) & set(truth_tokens)
  
  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0
  
  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)
  
  return round(2 * (prec * rec) / (prec + rec), 2)
  
def question_answer(context, question,answer):
  prediction = get_prediction(context,question)
  em_score = exact_match(prediction, answer)
  f1_score = compute_f1(prediction, answer)

  print(f'Question: {question}')
  print(f'Prediction: {prediction}')
  print(f'True Answer: {answer}')
  print(f'Exact match: {em_score}')
  print(f'F1 score: {f1_score}\n')

In [31]:
import jiwer

model.eval()
acc = []
wer = []
f1_scores=[]

for batch in tqdm(valid_loader):
    with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_true = batch['start_positions'].to(device)
        end_true = batch['end_positions'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        
        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)
        
        # Calculate accuracy
        acc.append(((start_pred == start_true).sum() / len(start_pred)).item())
        acc.append(((end_pred == end_true).sum() / len(end_pred)).item())
        
        # Calculate WER
        for i in range(len(start_true)):
            true_text = tokenizer.decode(input_ids[i][start_true[i]:end_true[i]])
            pred_text = tokenizer.decode(input_ids[i][start_pred[i]:end_pred[i]])
            if true_text.strip() == "":
                continue
            wer.append(jiwer.wer(true_text, pred_text))
            f1_scores.append(compute_f1(true_text, pred_text))
acc = sum(acc) / len(acc)
wer = sum(wer) / len(wer)

print(f'Accuracy: {acc:.4f}')
print(f'WER: {wer:.4f}')

print("\n\nT/P\tanswer_start\tanswer_end\n")
for i in range(len(start_true)):
    print(f"true\t{start_true[i]}\t{end_true[i]}\n"
          f"pred\t{start_pred[i]}\t{end_pred[i]}")

100%|██████████| 993/993 [09:40<00:00,  1.71it/s]

Accuracy: 0.0076
WER: 19.8983


T/P	answer_start	answer_end

true	59	59
pred	64	55
true	59	60
pred	64	55
true	59	59
pred	64	55


In [33]:
import numpy as np
overall_f1_score = np.mean(f1_scores)
print(overall_f1_score)

0.03714780390012757


In [34]:
def get_prediction(text, question):
  inputs = tokenizer.encode_plus(question, text, return_tensors='pt',max_length=512, truncation=True).to(device)

  outputs = model(**inputs)
  answer_start = torch.argmax(outputs[0])
  answer_end = torch.argmax(outputs[1]) + 1 

  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
  return answer


def prediction(contents,model):
  predAnswers={}
  for data in contents['data']:
    for txt in data['paragraphs']: 
      text = txt['context']
      for qa in txt['qas']:
        qid = qa['id']
        question = qa['question']
        predAnswers[qid]=get_prediction(text,question)
  return predAnswers

with open("spoken_test-v1.1_WER44.json", 'r') as j:
     valContents = json.loads(j.read())
     predAnswers=prediction(valContents,model)

with open('results.txt', 'w') as convert_file:
     convert_file.write(json.dumps(predAnswers))

In [35]:
def compute_wer(ground_truth, predicted):
    """
    Compute Word Error Rate (WER) between ground truth and predicted text.
    """
    # Convert both ground truth and predicted text to lists of words
    ground_truth_words = ground_truth.split()
    predicted_words = predicted.split()

    # Create a matrix of size (len(ground_truth_words) + 1) x (len(predicted_words) + 1)
    # and initialize the first row and column to 0, 1, 2, ..., len(predicted_words)
    matrix = [[j for j in range(len(predicted_words) + 1)]]
    matrix += [[i] + [0] * len(predicted_words) for i in range(1, len(ground_truth_words) + 1)]

    # Fill in the rest of the matrix
    for i in range(1, len(ground_truth_words) + 1):
        for j in range(1, len(predicted_words) + 1):
            if ground_truth_words[i - 1] == predicted_words[j - 1]:
                matrix[i][j] = matrix[i - 1][j - 1]
            else:
                substitution = matrix[i - 1][j - 1] + 1
                insertion = matrix[i][j - 1] + 1
                deletion = matrix[i - 1][j] + 1
                matrix[i][j] = min(substitution, insertion, deletion)

    # Return the WER, which is the bottom-right element of the matrix divided by the length of the ground truth
    return matrix[-1][-1] / len(ground_truth_words)


In [36]:
with open("spoken_test-v1.1_WER44.json", 'r') as j:
    valContents = json.loads(j.read())
    predAnswers = prediction(valContents, model)

totalWER = 0
numQuestions = 0

for qid, predAnswer in predAnswers.items():
    for data in valContents['data']:
        for txt in data['paragraphs']:
            for qa in txt['qas']:
                if qa['id'] == qid:
                    groundTruth = qa['answers'][0]['text']
                    wer = compute_wer(groundTruth, predAnswer)
                    totalWER += wer
                    numQuestions += 1
                    break
            else:
                continue
            break
        else:
            continue
        break

cumulativeWER = totalWER / numQuestions
print(f"Cumulative WER: {cumulativeWER}")


Cumulative WER: 23.005290791308266


In [37]:
with open("spoken_test-v1.1_WER54.json", 'r') as j:
    valContents = json.loads(j.read())
    predAnswers = prediction(valContents, model)

totalWER = 0
numQuestions = 0

for qid, predAnswer in predAnswers.items():
    for data in valContents['data']:
        for txt in data['paragraphs']:
            for qa in txt['qas']:
                if qa['id'] == qid:
                    groundTruth = qa['answers'][0]['text']
                    wer = compute_wer(groundTruth, predAnswer)
                    totalWER += wer
                    numQuestions += 1
                    break
            else:
                continue
            break
        else:
            continue
        break

cumulativeWER = totalWER / numQuestions
print(f"Cumulative WER: {cumulativeWER}")


Cumulative WER: 24.920225818046


In [38]:
context = """Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, 
          songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing 
          and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. 
          Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. 
          Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, 
          earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy"."""


questions = ["For whom the passage is talking about?",
             "When did Beyonce born?",
             "Where did Beyonce born?",
             "What is Beyonce's nationality?",
             "Who was the Destiny's group manager?",
             "What name has the Beyoncé's debut album?",
             "How many Grammy Awards did Beyonce earn?",
             "When did the Beyoncé's debut album release?",
             "Who was the lead singer of R&B girl-group Destiny's Child?"]

answers = ["Beyonce Giselle Knowles - Carter", "September 4, 1981", "Houston, Texas", 
           "American", "Mathew Knowles", "Dangerously in Love", "five", "2003", 
           "Beyonce Giselle Knowles - Carter"]

for question, answer in zip(questions, answers):
  question_answer(context, question, answer)

Question: For whom the passage is talking about?
Prediction: ##ɪ / bee - yon - say ) ( born september 4, 1981 ) is an american singer, songwriter, record producer and actress. born and raised in houston
True Answer: Beyonce Giselle Knowles - Carter
Exact match: False
F1 score: 0

Question: When did Beyonce born?
Prediction: ##ɪ / bee - yon - say ) ( born september 4, 1981 ) is an american singer, songwriter, record producer and actress. born and raised in houston
True Answer: September 4, 1981
Exact match: False
F1 score: 0.25

Question: Where did Beyonce born?
Prediction: ##ɪ / bee - yon - say ) ( born september 4, 1981 ) is an american singer, songwriter, record producer and actress. born and raised in houston
True Answer: Houston, Texas
Exact match: False
F1 score: 0.09

Question: What is Beyonce's nationality?
Prediction: ##ɪ / bee - yon - say ) ( born september 4, 1981 ) is an american singer, songwriter, record producer and actress. born and raised in houston
True Answer: America

In [ ]:
context = """Athens is the capital and largest city of Greece. Athens dominates the Attica region and is one of the world's oldest cities, 
             with its recorded history spanning over 3,400 years and its earliest human presence starting somewhere between the 11th and 7th millennium BC.
             Classical Athens was a powerful city-state. It was a center for the arts, learning and philosophy, and the home of Plato's Academy and Aristotle's Lyceum.
             It is widely referred to as the cradle of Western civilization and the birthplace of democracy, largely because of its cultural and political impact on the European continent—particularly Ancient Rome.
             In modern times, Athens is a large cosmopolitan metropolis and central to economic, financial, industrial, maritime, political and cultural life in Greece. 
             In 2021, Athens' urban area hosted more than three and a half million people, which is around 35% of the entire population of Greece.
             Athens is a Beta global city according to the Globalization and World Cities Research Network, and is one of the biggest economic centers in Southeastern Europe. 
             It also has a large financial sector, and its port Piraeus is both the largest passenger port in Europe, and the second largest in the world."""

questions = ["Which is the largest city in Greece?",
             "For what was the Athens center?",
             "Which city was the home of Plato's Academy?"]

answers = ["Athens", "center for the arts, learning and philosophy", "Athens"]

for question, answer in zip(questions, answers):
  question_answer(context, question, answer)
     

Question: Which is the largest city in Greece?
Prediction: piraeus
True Answer: Athens
Exact match: False
F1 score: 0

Question: For what was the Athens center?
Prediction: greece
True Answer: center for the arts, learning and philosophy
Exact match: False
F1 score: 0

Question: Which city was the home of Plato's Academy?
Prediction: athens is the capital and largest city of greece. athens dominates the attica region and is one of the world ' s oldest cities, with its recorded history spanning over 3, 400 years and its earliest human presence starting somewhere between the 11th and 7th millennium bc. classical athens was a powerful city - state. it was a center for the arts, learning and philosophy, and the home of plato ' s academy and aristotle ' s lyceum. it is widely referred to as the cradle of western civilization and the birthplace of democracy, largely because of its cultural and political impact on the european continent — particularly ancient rome. in modern times, athens is 

In [39]:
context = """Angelos Poulis was born on 8 April 2001 in Nicosia, Cyprus. He is half Cypriot and half Greek. 
            He is currently studying at the Department of Informatics and Telecommunications of the University of Athens in Greece. 
            His scientific interests are in the broad field of Artificial Intelligence and he loves to train neural networks! 
            Okay, I'm Angelos and I'll stop talking about me right now."""

questions = ["When did Angelos born?",
             "In what university is Angelos studying now?",
             "What is Angelos' nationality?",
             "What are his scientific interests?",
             "What I will do right now?"]

answers = ["8 April 2001", "University of Athens", 
           "half Cypriot and half Greek", "Artificial Intelligence", 
           "stop talking about me"]

for question, answer in zip(questions, answers):
  question_answer(context, question, answer)

Question: When did Angelos born?
Prediction: half cypriot
True Answer: 8 April 2001
Exact match: False
F1 score: 0

Question: In what university is Angelos studying now?
Prediction: half cypriot
True Answer: University of Athens
Exact match: False
F1 score: 0

Question: What is Angelos' nationality?
Prediction: half cypriot
True Answer: half Cypriot and half Greek
Exact match: False
F1 score: 0.57

Question: What are his scientific interests?
Prediction: half cypriot
True Answer: Artificial Intelligence
Exact match: False
F1 score: 0

Question: What I will do right now?
Prediction: half cypriot
True Answer: stop talking about me
Exact match: False
F1 score: 0

